In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

**Checking for Duplicates**

In [ ]:
dups = df_train.duplicated()

In [ ]:
print('Number of duplicate rows = %d' % (dups.sum()))

**Removing Outliers**

In [ ]:
import seaborn as sns

In [ ]:
len(df_train.columns)

In [ ]:
for column in df_train.columns:
    sns.boxplot(x=df_train[column])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df_train.corr()

In [ ]:
def remove_outlier(col):
    sorted(col)
    Q1,Q3=np.percentile(col,[25,75])
    IQR=Q3-Q1
    lower_range= Q1-(1.5 * IQR)
    upper_range= Q3+(1.5 * IQR)
    return lower_range, upper_range

In [ ]:
for column in df_train.columns:
    lr,ur=remove_outlier(df_train[column])
    df_train[column]=np.where(df_train[column]>ur,ur,df_train[column])
    df_train[column]=np.where(df_train[column]<lr,lr,df_train[column])

**Feature Selection**

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2, f_regression

In [ ]:
# Copy all the predictor variables into X dataframe
X = df_train.drop('loss', axis=1)

# Copy target into the y dataframe. 
y = df_train[['loss']]

In [ ]:
# Split X and y into training and test set in 75:25 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25 , random_state=1)

In [ ]:
fs = SelectKBest(score_func=f_regression, k=15)

In [ ]:
fs.fit(X_train, y_train)

In [ ]:
X_train_fs = fs.transform(X_train)

In [ ]:
X_train_fs.shape

In [ ]:
df_test.head()

In [ ]:
X_test_fs = fs.transform(X_test)

In [ ]:
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
import pandas_profiling
from plotnine import *

In [ ]:
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.show()

**Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# invoke the LinearRegression function and find the bestfit model on training data
regression_model = LinearRegression()
regression_model.fit(X_train_fs, y_train)

In [ ]:
regression_model.coef_ #[0][0] #[0][2]

In [ ]:
regression_model.score(X_train_fs, y_train)

In [ ]:
# R square on testing data
regression_model.score(X_test_fs, y_test)

In [ ]:
#RMSE on Training data
predicted_train = regression_model.fit(X_train_fs, y_train).predict(X_train_fs)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
yhat = regression_model.predict(X_test_fs)

In [ ]:
mae = mean_absolute_error(y_test, yhat)

In [ ]:
print('MAE: %.3f' % mae)

In [ ]:
from sklearn import metrics

In [ ]:
np.sqrt(metrics.mean_squared_error(y_train, predicted_train))

In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, yhat))

In [ ]:
X_test_pd = fs.transform(df_test)

In [ ]:
predicted_test = regression_model.predict(X_test_pd)

In [ ]:
predicted_test

In [ ]:
df_test['loss'] = predicted_test

In [ ]:
df_test2 = df_test[['id','loss']]

In [ ]:
df_test2.head()

In [ ]:
df_test2.to_csv('test.csv')
df_test2.to_csv('submission.csv')

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error,mean_squared_log_error,make_scorer
from sklearn.pipeline import make_pipeline

In [ ]:
mod=RandomForestRegressor(n_estimators=50)

In [ ]:
model=make_pipeline(mod)

In [ ]:
print(model)

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV

In [ ]:
kfold=KFold(n_splits=5)

In [ ]:
model.fit(X_train_fs,y_train)

In [ ]:
score=cross_val_score(model,X_train_fs,y_train,cv=kfold)

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV

In [ ]:
kfold=KFold(n_splits=5)

In [ ]:
model.fit(X_train_fs,y_train)

In [ ]:
score=cross_val_score(model,X_train_fs,y_train,cv=kfold)

In [ ]:
yp=model.predict(X_test_fs)

In [ ]:
print('Match not Found')

In [ ]:
print(r2_score(y_test,yp),'Rsquare')

In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, yp))

In [ ]:
print(mean_squared_error(y_test,yp), 'MSE')

In [ ]:
yp2=model.predict(X_test_pd)

In [ ]:
yp2

In [ ]:
df_test['lossrf'] = yp2

In [ ]:
df_test3 = df_test[['id','lossrf']]

In [ ]:
df_test3.rename(columns={"id": "id","lossrf":"loss"})

In [ ]:
df_test3.to_csv('submission_rf.csv')

**XG Boost Regressor**

In [ ]:
import xgboost as xgb

In [ ]:
print(xgb.__version__)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X_train_fs,y_train)

In [ ]:
y_test_preds = xg_reg.predict(X_test_fs)

In [ ]:
y_train_preds = xg_reg.predict(X_train_fs)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train, y_train_preds))
print("RMSE: %f" % (rmse))

In [ ]:
np.sqrt(metrics.mean_squared_error(y_train, y_train_preds))

In [ ]:
data_dmatrix = xgb.DMatrix(data=X_train_fs,label=y_train)

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
y_test_preds = xg_reg.predict(X_test_pd)

In [ ]:
df_test['lossxgb'] = y_test_preds

In [ ]:
df_test4 = df_test[['id','lossxgb']]

In [ ]:
df_test4.rename(columns={"id": "id","lossxgb":"loss"})

In [ ]:
df_test4.to_csv('submission_xgb.csv')

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
data_new = df_train2.apply(zscore)